In [1]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, FloatType
import pandas as pd
import numpy as np

In [2]:
spark = SparkSession\
    .builder\
    .appName("pyspark_app")\
    .getOrCreate()

In [4]:
sc = spark.sparkContext

In [7]:
mu, sigma = 2, 0.5
num_elements=1000
v = np.random.normal(mu, sigma, num_elements)
rdd1 = sc.parallelize(v)

In [8]:
rdd1.count()

1000

In [9]:
def mult(x): 
    return x * np.array([2])   # double each element
rdd2 = rdd1.map(mult).map(lambda x: (float(x),))

In [13]:
rdd2.toDF().show(10)

+------------------+
|                _1|
+------------------+
| 3.670658395702397|
| 5.529276450400716|
|3.7459756413416434|
| 4.102683141845928|
|3.3512970146476384|
| 3.504555655585047|
| 5.463597364507127|
|3.8625419495098736|
|3.1614949296528905|
| 2.694182415265402|
+------------------+
only showing top 10 rows



In [27]:
schema = StructType([StructField("value", FloatType(), True)])
df1 = spark.createDataFrame(rdd2, schema)
df1.registerTempTable("test")
def square(x): 
    return round(x * x, 5)
spark.udf.register("squared", square)
df2 = spark.sql("SELECT squared(value) squared FROM test")

In [28]:
df2.show(10)

+--------+
| squared|
+--------+
|13.47373|
| 30.5729|
|14.03233|
|16.83201|
|11.23119|
|12.28191|
| 29.8509|
|14.91923|
| 9.99505|
| 7.25862|
+--------+
only showing top 10 rows



In [29]:
pdf2 = df2.toPandas()

In [30]:
pdf2.head()

,squared
0,13.47373
1,30.5729
2,14.03233
3,16.83201
4,11.23119


In [31]:
pdf1 = df1.toPandas()

In [32]:
pdf1["squared"] = pdf1["value"].apply(lambda x: x*x)

In [33]:
pdf1.head()

,value,squared
0,3.670658,13.473733
1,5.529276,30.572897
2,3.745976,14.032334
3,4.102683,16.832008
4,3.351297,11.231191
